In [9]:
#This part is not needed for now

# Import standard libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

# Import TF layers
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D

# Import TF utilities
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# Import TF pretrained models
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.nasnet import NASNetMobile

Below we train a simple model in supervised learning without data augmentation. We check its performance on the cifar10 dataset. It will be our un-noised teacher model. And we will compare its performance with 100 epochs to the semi supervised self-learning model in the following block

In [17]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

batch_size = 200
num_classes = 10
epochs = 100
data_augmentation = False
#num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
teacher_name = 'keras_cifar10_trained_teacher.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

teacher = Sequential()
teacher.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
teacher.add(Activation('relu'))
teacher.add(Conv2D(32, (3, 3)))
teacher.add(Activation('relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))
teacher.add(Dropout(0.25))

teacher.add(Conv2D(64, (3, 3), padding='same'))
teacher.add(Activation('relu'))
teacher.add(Conv2D(64, (3, 3)))
teacher.add(Activation('relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))
teacher.add(Dropout(0.25))

teacher.add(Flatten())
teacher.add(Dense(512))
teacher.add(Activation('relu'))
#model.add(Dropout(0.5))
teacher.add(Dense(num_classes))
teacher.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Compiling the model using RMSprop
teacher.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

#Training the model

if not data_augmentation:
    print('Not using data augmentation.')
    teacher.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2,
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    teacher.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_split=0.2,
                        workers=4)
    
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
teacher_path = os.path.join(save_dir, teacher_name)
teacher.save(teacher_path)
print('Saved trained model at %s ' % teacher_path)

# Score trained model.
scores = teacher.evaluate(x_test, y_test, verbose=1)
print('Supervised learning model with '+str(epochs)+'epochs \n')
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Not using data augmentation.
Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 66s 2ms/step - loss: 1.9824 - accuracy: 0.2788 - val_loss: 1.7956 - val_accuracy: 0.3510
Epoch 2/100
40000/40000 [==============================] - 65s 2ms/step - loss: 1.7091 - accuracy: 0.3855 - val_loss: 1.6613 - val_accuracy: 0.4083
Epoch 3/100
40000/40000 [==============================] - 65s 2ms/step - loss: 1.6102 - accuracy: 0.4207 - val_loss: 1.5563 - val_accuracy: 0.4370
Epoch 4/100
40000/40000 [==============================] - 65s 2ms/step - loss: 1.5374 - accuracy: 0.4475 - val_loss: 1.5029 - val_accuracy: 0.4616
Epoch 5/100
40000/40000 [==============================] - 66s 2ms/step - loss: 1.4740 - accuracy: 0.4713 - val_loss: 1.4354 - val_accuracy: 0.4915
Epoch 6/100
40000/40000 [==============================] - 66s 2ms/step - loss: 1.4250 - accuracy: 0.4887 - 

40000/40000 [==============================] - 68s 2ms/step - loss: 0.5219 - accuracy: 0.8185 - val_loss: 0.7801 - val_accuracy: 0.7392
Epoch 56/100
40000/40000 [==============================] - 67s 2ms/step - loss: 0.5038 - accuracy: 0.8271 - val_loss: 0.7547 - val_accuracy: 0.7493
Epoch 57/100
40000/40000 [==============================] - 66s 2ms/step - loss: 0.4963 - accuracy: 0.8303 - val_loss: 0.7625 - val_accuracy: 0.7448
Epoch 58/100
40000/40000 [==============================] - 67s 2ms/step - loss: 0.4825 - accuracy: 0.8321 - val_loss: 0.7603 - val_accuracy: 0.7493
Epoch 59/100
40000/40000 [==============================] - 67s 2ms/step - loss: 0.4721 - accuracy: 0.8372 - val_loss: 0.7483 - val_accuracy: 0.7505
Epoch 60/100
40000/40000 [==============================] - 69s 2ms/step - loss: 0.4575 - accuracy: 0.8446 - val_loss: 0.7479 - val_accuracy: 0.7537
Epoch 61/100
40000/40000 [==============================] - 72s 2ms/step - loss: 0.4458 - accuracy: 0.8461 - val_loss: 

Below we study the semi supervised learning method as described in the article, and study the impact of the ratio of true labels to pseudo labels on accuracy and loss. 
We do a loop over different ratios of label to unlabelled data (rate). And each loop does the following:
1) Train un-noised model (teacher) on labeled data only 
2) Ten cycles of: un-noised model (teacher)->predict hard pseudolabel->training 10 epochs for noised model (student=teacher+dropout noise) on labeled+pseudo labeled->new weights. 

We check rates from 0.1 to 50. The student model will be the teacher model noised by a Dropout(0.5) before the last layer

In [16]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

#batch_size = 32
num_classes = 10
#epochs = 50
#data_augmentation = False
#num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
teacher_name = 'keras_cifar10_trained_teacher.h5'

teacher_path = os.path.join(save_dir, teacher_name)

# Load the cifar10 data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


#making un-noised teacher model
teacher = Sequential()
teacher.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
teacher.add(Activation('relu'))
teacher.add(Conv2D(32, (3, 3)))
teacher.add(Activation('relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))
teacher.add(Dropout(0.25))

teacher.add(Conv2D(64, (3, 3), padding='same'))
teacher.add(Activation('relu'))
teacher.add(Conv2D(64, (3, 3)))
teacher.add(Activation('relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))
teacher.add(Dropout(0.25))

teacher.add(Flatten())
teacher.add(Dense(512))
teacher.add(Activation('relu'))
#model.add(Dropout(0.5)) #this will be uncommented for the noised student model
teacher.add(Dense(num_classes))
teacher.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Compile the teacher model using RMSprop
teacher.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255



#noised student model

student = Sequential()
student.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
student.add(Activation('relu'))
student.add(Conv2D(32, (3, 3)))
student.add(Activation('relu'))
student.add(MaxPooling2D(pool_size=(2, 2)))
student.add(Dropout(0.25))

student.add(Conv2D(64, (3, 3), padding='same'))
student.add(Activation('relu'))
student.add(Conv2D(64, (3, 3)))
student.add(Activation('relu'))
student.add(MaxPooling2D(pool_size=(2, 2)))
student.add(Dropout(0.25))

student.add(Flatten())
student.add(Dense(512))
student.add(Activation('relu'))
student.add(Dropout(0.5))
student.add(Dense(num_classes))
student.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Compiling the model using RMSprop
student.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])





# rate= n_true/n_pseudo
rate=np.array([0.1,0.2,0.3,0.5,0.75,1,2,3,4,5,10,20,30,50])
n_total=x_train.shape[0]

# total number of train images (n_total) = number of true label images (n_true) + number of pseudo label images (n_pseudo)
#                                        = n_pseudo(rate+1)
# n_pseudo = n_total/(rate+1); n_true=n_total-n_pseudo

#loop over rate values in order to find the optimal rate value for the self-learning semi supervised learning, 
#ie one that will maximize accuracy
for r in rate:
    print("rate="+str(r)+":\n")
    n_pseudo=n_total/(1+r)
    n_true=n_total-n_pseudo
    mask_true=np.random.choice(int(n_total),int(n_true),replace=False) #generating n_true integers between 0 and n_total-1
    mask_pseudo=[item for item in range(n_total) if item not in mask_true] #all the other numbers between 0 and n_total-1 which are not in mask_true
    mask_pseudo=np.array(mask_pseudo)
    x_true=x_train[mask_true[:]] #x for the labeled data
    y_true=y_train[mask_true[:]] #y for the labeled data
    x_pseudo=x_train[mask_pseudo[:]] #x for the unlabeled data (pseudo)
    #training teacher model on labeled data with validation split of 0.2
    training=teacher.fit(x_true,y_true,validation_split=0.2,
                            epochs=30,batch_size=200,verbose=0)
    #evaluating teacher model on test data    
    scores=teacher.evaluate(x_test,y_test,verbose=0)
    print("Original model with labelled data only predicting on test data: ",scores[1])

    x_true_pseudo=np.concatenate([x_true,x_pseudo]) #concatenating x for labeled and unlabeled data
    print('x_true_pseudo.shape: ',x_true_pseudo.shape)
    prediction=teacher.predict_classes(x_pseudo) #predicting labels on unlabeled data
    y_pseudo=keras.utils.to_categorical(prediction, num_classes)
    y_true_pseudo=np.concatenate([y_true,y_pseudo]) #concatenating y for labeled and pseudo labeled
    print('y_true_pseudo.shape: ', y_true_pseudo.shape)
    for i in range(10): 
        # 10 loops of 10 epochs of noised student training for labeled and pseudo labeled data (step 3 in article)
        # followed by generating predictions on unlabeled data with the teacher model (=un-noised student)
        # which uses the weights of the trained noised student (noise does not change the weights structure of models) (step 2 in article)
        print(i)
        training=student.fit(x_true_pseudo,y_true_pseudo,validation_split=0.2,
                             epochs=10,batch_size=200,verbose=0)
        # Save weights
        student.save_weights(teacher_path)
        # Load weights for teacher model (un-noised)
        teacher.load_weights(teacher_path)
        prediction=teacher.predict_classes(x_pseudo)
        scores=teacher.evaluate(x_test,y_test,verbose=0) #evaluating model on test data
        print('iteration: ',i)
        print('Test loss:', scores[0])
        print('Test accuracy:', scores[1])
        y_pseudo=keras.utils.to_categorical(prediction, num_classes)
        y_true_pseudo=np.concatenate([y_true,y_pseudo]) #new y_true_pseudo to be used in next loop
        




x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
rate=0.1:

Original model with labelled data only predicting on test data:  0.4259999990463257
x_true_pseudo.shape:  (50000, 32, 32, 3)
y_true_pseudo.shape:  (50000, 10)
0
iteration:  0
Test loss: 2.3814638202667235
Test accuracy: 0.40630000829696655
1
iteration:  1
Test loss: 2.481840321731567
Test accuracy: 0.3961000144481659
2
iteration:  2
Test loss: 2.5549630054473877
Test accuracy: 0.39399999380111694
3
iteration:  3
Test loss: 2.659085976409912
Test accuracy: 0.41119998693466187
4
iteration:  4
Test loss: 2.6760872135162352
Test accuracy: 0.4178999960422516
5
iteration:  5
Test loss: 2.606792931365967
Test accuracy: 0.4214000105857849
6
iteration:  6
Test loss: 2.630066015625
Test accuracy: 0.4205000102519989
7
iteration:  7
Test loss: 2.694792952346802
Test accuracy: 0.414000004529953
8
iteration:  8
Test loss: 2.6629435035705566
Test accuracy: 0.4052000045776367
9
iteration:  9
Test loss: 2.55188693542480

iteration:  3
Test loss: 0.638658087682724
Test accuracy: 0.7925000190734863
4
iteration:  4
Test loss: 0.639220199394226
Test accuracy: 0.7942000031471252
5
iteration:  5
Test loss: 0.6503369867324829
Test accuracy: 0.7918999791145325
6
iteration:  6
Test loss: 0.6423421550750732
Test accuracy: 0.7924000024795532
7
iteration:  7
Test loss: 0.6258201817035675
Test accuracy: 0.7961999773979187
8
iteration:  8
Test loss: 0.6589927075386047
Test accuracy: 0.796999990940094
9
iteration:  9
Test loss: 0.6539146895885467
Test accuracy: 0.7936999797821045
rate=5.0:

Original model with labelled data only predicting on test data:  0.7867000102996826
x_true_pseudo.shape:  (50000, 32, 32, 3)
y_true_pseudo.shape:  (50000, 10)
0
iteration:  0
Test loss: 0.631142573595047
Test accuracy: 0.7993000149726868
1
iteration:  1
Test loss: 0.6383090171337128
Test accuracy: 0.7996000051498413
2
iteration:  2
Test loss: 0.6523478969573975
Test accuracy: 0.7944999933242798
3
iteration:  3
Test loss: 0.6502401

We can see that the range of ratios of labeled/unlabeled data of 3 to 30 look optimal. The accuracy on test data with the un-noised model used in supervised learning for 100 epochs is 0.7472, while the self-learning semi supervised learning algorithm with the same model and number of epochs give an accuracy in excess of 0.795. Noise was added on the model with a DropOut(0.5) on the layer before the last layer.